In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"
3,290620657799159809,Anne Hathaway has got me living. #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': '_NicoleEdwards', 'id': 144430..."
4,290620657778188288,Jennifer Lopez's lace dress? Thoughts? #Golden...,2013-01-14 00:45:38,"{'screen_name': 'lolaogunnaike', 'id': 134953223}"
5,290620657719455745,Podrán criticar a #Adele de su moda y su maniq...,2013-01-14 00:45:38,"{'screen_name': 'SabyOnford', 'id': 404554221}"
6,290620657715273728,"US, Weakly #GoldenGlobes",2013-01-14 00:45:38,"{'screen_name': 'alejandraoleary', 'id': 45748..."
7,290620657715253248,"RT @BillMc7: ""Wait. What's that smell?!"" (ever...",2013-01-14 00:45:38,"{'screen_name': 'xtinavickers', 'id': 92139052}"
8,290620657706872832,Hugh Jackman is so awesome!!! #goldenglobes,2013-01-14 00:45:38,"{'screen_name': 'Boaz_173', 'id': 31504713}"
9,290620657560084480,It was awkward. RT @hollywoodhwife: They cut t...,2013-01-14 00:45:38,"{'screen_name': 'KateSpencer1', 'id': 21571382}"


In [3]:
len(df)

174643

In [4]:
data = df['text']
data[:10]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
5    Podrán criticar a #Adele de su moda y su maniq...
6                             US, Weakly #GoldenGlobes
7    RT @BillMc7: "Wait. What's that smell?!" (ever...
8          Hugh Jackman is so awesome!!! #goldenglobes
9    It was awkward. RT @hollywoodhwife: They cut t...
Name: text, dtype: object

In [5]:
data[1]

"What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes"

In [6]:
import re

In [7]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [8]:
# test
for line in data[:10]:
    print(line)
    print(cleanse(line))
    print(line.split())
    print()

JLo's dress! #eredcarpet #GoldenGlobes
JLo's dress  #eredcarpet #GoldenGlobes
["JLo's", 'dress!', '#eredcarpet', '#GoldenGlobes']

What's making Sofia Vergara's boobs stay like that? Magic? Witchcraft? #GoldenGlobes
What's making Sofia Vergara's boobs stay like that  Magic  Witchcraft  #GoldenGlobes
["What's", 'making', 'Sofia', "Vergara's", 'boobs', 'stay', 'like', 'that?', 'Magic?', 'Witchcraft?', '#GoldenGlobes']

RT @FabSugar: Kerry Washington is EVERYTHING. Dying over her Miu Miu gown! @goldenglobes #goldenglobes http://t.co/3feH7MvO
RT @FabSugar  Kerry Washington is EVERYTHING  Dying over her Miu Miu gown  @goldenglobes #goldenglobes http   t co 3feH7MvO
['RT', '@FabSugar:', 'Kerry', 'Washington', 'is', 'EVERYTHING.', 'Dying', 'over', 'her', 'Miu', 'Miu', 'gown!', '@goldenglobes', '#goldenglobes', 'http://t.co/3feH7MvO']

Anne Hathaway has got me living. #GoldenGlobes
Anne Hathaway has got me living  #GoldenGlobes
['Anne', 'Hathaway', 'has', 'got', 'me', 'living.', '#GoldenGlobes

In [9]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [10]:
def identify_entities(text, verbose=False):
    ents = list(nlp(text).ents)
    if verbose:
        person_tags = {}
        tags = {}
        for ent in ents:    
            entity = ' '.join(t.orth_ for t in ent).strip()
            if entity not in tags:
                tags[entity]=[ent.label_]
            if ent.label_ == 'PERSON' and entity not in person_tags:
                person_tags[entity]=[ent.label_]
        print('person tags:', person_tags)
        return tags
    else:
        return [' '.join(t.orth_ for t in ent).strip() for ent in ents]

In [11]:
# test
text = "“We’re going to keep things loose,” said Amy Poehler of her and co-host Tina Fey’s plan for the evening. #GoldenGlobes http://t.co/o40g5LGq“"
print('tags:', identify_entities(text, True))
print(identify_entities(text, False))

person tags: {'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON']}
tags: {'Amy Poehler': ['PERSON'], 'Tina Fey ’s': ['PERSON'], 'the evening': ['TIME'], 'GoldenGlobes': ['MONEY']}
['Amy Poehler', 'Tina Fey ’s', 'the evening', 'GoldenGlobes']


In [12]:
def find_host(verbose=False):
    pattern = re.compile(r'host')
    entity_dict = {}
    
    num = 0
    for line in data:
        match = re.findall(pattern, cleanse(line).lower())
        if match:
            entities = identify_entities(cleanse(line), False)

            if verbose:
                # print the first 5 occurrences
                if num < 5:
                    print(match)
                    print(line)
                    print(entities)
                    print()

            for entity in entities:
                if len(entity) > 1:
                    if entity not in entity_dict:
                        entity_dict[entity] = 1
                    else:
                        entity_dict[entity] += 1
            num += 1
            
    print('num of matches:', num)
    return entity_dict

In [13]:
%%time
entity_dict = find_host(verbose=True)

['host']
Looking forward to watching Tina Fey and Amy Poehler host the #GoldenGlobes
['Tina Fey', 'Amy Poehler']

['host']
RT @goldenglobes: It's our hosts Tina Fey and Amy Poehler! #goldenglobes #redcarpet http://t.co/8lqC3ocQ
['RT @goldenglobes', 'Tina Fey', 'Amy Poehler', '#']

['host']
RT @washingtonpost: Tonight's dual hosting duties represent the culmination of a decade of Amy and Tina partnerships. http://t.co/scSThrn5 #GoldenGlobes
['RT @washingtonpost', 'Tonight', 'a decade', 'Amy', 'Tina', '']

['host']
RT @MovieMayor: My green suede tuxedo pinching a bit here at the Velvet Rope Awards honoring best in crowd control. Topo Gigio + I hosting #GoldenGlobes
['RT @MovieMayor', 'the Velvet Rope Awards', '', 'Topo Gigio', 'GoldenGlobes']

['host']
RT @accesshollywood: Tina Fey &amp; Amy Poehler Talk #GoldenGlobes Hosting, Drinking Game  http://t.co/1mIvq362
['RT @accesshollywood', 'Amy Poehler Talk', 'GoldenGlobes Hosting', 'Drinking Game', '']

num of matches: 3109
CPU times: user 

In [14]:
# list the first 3 keys are not enough
list(entity_dict.keys())[:3]

['Tina Fey', 'Amy Poehler', 'RT @goldenglobes']

In [15]:
top_100 = sorted(entity_dict.items(), key=lambda pair: pair[1])[-100:][::-1]
top_100

[('GoldenGlobes', 808),
 ('Will Ferrell', 703),
 ('Kristen Wiig', 671),
 ('Amy Poehler', 618),
 ('Amy', 537),
 ('Tina', 425),
 ('next year', 378),
 ('Tina Fey', 301),
 ('Oscars', 278),
 ('RT @PerezHilton', 237),
 ('RT @nbcsnl', 166),
 ('Golden Globes', 124),
 ('RT @eonline', 114),
 ('2014', 98),
 ('Quentin Tarantino', 97),
 ('Fey', 89),
 ('Ever', 89),
 ('Best   Hosts', 88),
 ('RT @EW', 71),
 ('RT', 69),
 ('GoldenGlobes 2014', 63),
 ('Scratch', 63),
 ('Would', 62),
 ('the Golden Globes', 61),
 ('# GoldenGlobes', 55),
 ('Opening Monologue', 54),
 ('Paul Rudd', 53),
 ('tonight', 49),
 ('Golden Globes Hosts Tina', 48),
 ('Amy Poelher', 48),
 ('two', 47),
 ('Thoroughly', 46),
 ('SNL', 46),
 ('RT @HechosTHG', 44),
 ('Hollywood', 44),
 ('Can Tina', 42),
 ("next year 's", 41),
 ('RT @piersmorgan', 38),
 ('RT @fuggirls', 38),
 ('RT @laurahertzfeld', 38),
 ('this Golden Globes   Tina', 37),
 ('# GoldenGlobes #', 32),
 ('Iran', 31),
 ('Please', 31),
 ('Academy', 28),
 ('RT @jianghomeshi', 27),
 (

In [16]:
# pip3 install python-Levenshtein for 4-10x speedup
from fuzzywuzzy import fuzz

# test
print(fuzz.ratio('Tina Fey', 'Tina'))
print(fuzz.ratio('Amy Poehler', 'Amy'))
print(fuzz.ratio('Golden Globes', 'golden globes'))

67
43
85


In [17]:
# remove golden globes from names
import pprint
names = [pair[0] for pair in top_100]
golden_globes = [name for name in names if fuzz.ratio(name.lower(), 'golden globes') > 50]
pprint.pprint(golden_globes)

['GoldenGlobes',
 'Golden Globes',
 'GoldenGlobes 2014',
 'the Golden Globes',
 '# GoldenGlobes',
 'Golden Globes Hosts Tina',
 'this Golden Globes   Tina',
 '# GoldenGlobes #',
 'Amy Poehler on Hosting Golden Globes',
 '@GoldenGlobes']


In [18]:
for name in golden_globes:
    del entity_dict[name]
top_20 = sorted(entity_dict.items(), key=lambda pair: pair[1])[-20:][::-1]
top_20

[('Will Ferrell', 703),
 ('Kristen Wiig', 671),
 ('Amy Poehler', 618),
 ('Amy', 537),
 ('Tina', 425),
 ('next year', 378),
 ('Tina Fey', 301),
 ('Oscars', 278),
 ('RT @PerezHilton', 237),
 ('RT @nbcsnl', 166),
 ('RT @eonline', 114),
 ('2014', 98),
 ('Quentin Tarantino', 97),
 ('Fey', 89),
 ('Ever', 89),
 ('Best   Hosts', 88),
 ('RT @EW', 71),
 ('RT', 69),
 ('Scratch', 63),
 ('Would', 62)]

In [19]:
names = [pair[0] for pair in top_20]
names_clusters = []

for name in names:
#     print('name to reduce:', name)
    cluster = [name]
    names_to_reduce = names[:]
    names_to_reduce.remove(name)
    
    for i in names_to_reduce:
        ratio = fuzz.ratio(name.lower(), i.lower())
        if ratio > 50 or re.findall(name.lower(), i.lower()) or re.findall(i.lower(), name.lower()):
            cluster.append(i)
            
    if len(cluster) > 1:
        names_clusters.append(cluster)
#     print(cluster)
#     print('\n')
names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
names_clusters_reduced

[['Fey', 'Tina', 'Tina Fey'],
 ['Fey', 'Tina Fey'],
 ['Tina', 'Tina Fey'],
 ['RT', 'RT @EW', 'RT @eonline'],
 ['RT', 'RT @PerezHilton', 'RT @eonline'],
 ['Amy', 'Amy Poehler'],
 ['RT', 'RT @EW', 'RT @PerezHilton', 'RT @eonline', 'RT @nbcsnl'],
 ['RT', 'RT @eonline', 'RT @nbcsnl']]

In [20]:
e = entity_dict.copy()
for cluster in names_clusters_reduced:
    # select the longest name
    selected_name = max(cluster, key=len)
    cluster.remove(selected_name)
    for name in cluster:
        # if not deleted in previous cases
        if name in e and selected_name in e:
            e[selected_name] += e[name]
            del e[name]

In [21]:
top_10 = sorted(e.items(), key=lambda pair: pair[1])[-10:][::-1]
top_10

[('Amy Poehler', 1155),
 ('Tina Fey', 815),
 ('Will Ferrell', 703),
 ('Kristen Wiig', 671),
 ('RT @PerezHilton', 657),
 ('next year', 378),
 ('Oscars', 278),
 ('2014', 98),
 ('Quentin Tarantino', 97),
 ('Ever', 89)]

In [22]:
def remove_abbreviation(text):
    if text.endswith('s'):
        return text[:-2].strip()
    return text

# test
print(remove_abbreviation("Tina 's"))

Tina


In [23]:
# top 3 inferences for host
[name[0] for name in top_10][:3]

['Amy Poehler', 'Tina Fey', 'Will Ferrell']